Make sure you are running this in a virtual environment! Otherwise you may cause problems to your main device. 

Look up how to create a virtual environment if you don't know how. Otherwise you can just run this in google colab. 

Then in your virtual environment run the following lines:

***

pip install jupyter

pip install ipykernel

python -m ipykernel install --user --name=myenv --display-name "Python (myenv)"

pip install tensorflow

pip install shap

***

The lines will install different packages and libraries that you need for this notebook. I may have forgotten some in which case you can likely just pip install "name of package" in the same format as above.

Also, make sure to download all the files called here and every file in the 'data_files_for_data_construction' folder. Ideally you just download the whole github, but some of the CSV files are rather large.

***

Before we begin we have some standard python libraries to import that we will use throughout this notebook.

In [1]:
import pandas as pd
import time

There was no existing dataset that contained the data needed for this project. Thus first we must generate a synthetic dataset. The dataset will be generated based on a variety of real data, mappings between datasets, and artificially generated lists. 

First we import the Data class which contains all the data needed to generate the synthetic dataset.

Next we import the DataGenerator class for the CPU. Note that a version does exist that runs on the GPU.

In [2]:
from datafiles_for_data_construction.data import Data
from data_generation.data_generation_CPU import DataGenerator

Next, we instantiate the Data and DataGenerator classes. The Data class allows us to access all the data needed to generate the synthetic dataset and the DataGenerator class allows us to use the functions needed to generate the synthetic dataset.

In [3]:
data = Data()
data_generator = DataGenerator(data)

What does the data look like? Some of the data is a list of values. Some lists were generated synthetically, others were pulled from various sources. More information can be found in the README file. Here is a list of learning styles:

In [4]:
data.learning_style()["learning_style_list"]

['Visual', 'Auditory', 'Read/Write', 'Kinesthetic']

Some of the data is a dictionary. Some dictionaries map different lists together while others map lists to demographic statistics on how common each item is. This dictionary maps the learning styles to the percentage of people that have said style.

In [5]:
data.learning_style()["learning_style"]

{'Visual': 27.27, 'Auditory': 23.56, 'Read/Write': 21.16, 'Kinesthetic': 28.01}

Now we use the generate_synthetic_dataset function to create a dataset from all the data. This function has two inputs:
- number of samples (an integer) which tell the function how many 'students' we want in our dataset
- batch size (an integer) which tells the function how to split up the work to prevent overloading the computer.
You can change the values if you want to generate more or less data. Be careful as higher values for number of samples will lead to a longer runtime.

In [6]:
num_samples = 100 # You can change these values if you want
batch_size = 10 # Batch size should be about 1/10 of the number of samples

Now we call the function. Use the time library to see how long the generator takes.

In [7]:
start_time = time.time()
synthetic_data = data_generator.generate_synthetic_dataset(num_samples, batch_size)
end_time = time.time()
runtime = end_time - start_time
print(runtime)

20.10244607925415


'generate_synthetic_dataset' outputs a pandas dataframe. Lets look at the top 5 elements of the dataframe. You can look back at the README file to get a better sense of what each column contains and how it was generated.

In [8]:
synthetic_data.head(n=5) # Change n to larger numbers to see more rows of the dataframe

,first name,last name,ethnoracial group,gender,international status,socioeconomic status,learning style,gpa,student semester,major,previous courses,course types,course subjects,subjects of interest,extracurricular activities,career aspirations,future topics
0,Nima,Millon,Latino/a/x American,Female,Domestic,Higher income,[Kinesthetic],3.90,12,[Biomedical Engineering],"[Leadership Laboratory, Climate and Global Cha...","[[Laboratory, Lecture], [Lecture-Discussion, P...","[CMN, EALC, BADM, MATH, ACCY, LLS, PHYS, TAM, ...","[Environmental Science, Psychology, Biomedical...","[Sorority Council, Computer Science Club, Engi...","[Software Developer, Application and System So...","[Biology, Biomedical Engineering, Engineering,..."
1,Tywone,Liebner,European American or white,Male,Domestic,Middle income,[Read/Write],2.20,12,[Cosmetology Services And Culinary Arts],"[Professional Applications, Vocabulary Buildin...","[[Laboratory, Lecture], [Lecture, Practice], [...","[CMN, ATMS, VCM, PHYS, IE, EPSY, GGIS, IB, CHL...","[Political Science, Agricultural Sciences, Che...","[Environmental Science Club, Hospitality Manag...","[Farmer, Rancher, and Other Agricultural Manag...","[Finance, Marketing, Culinary Arts, Health and..."
2,Hollis,Lacoe,European American or white,Female,Domestic,Middle income,[Kinesthetic],2.27,5,"[Interdisciplinary Social Sciences, Neuroscience]","[Undergraduate Open Seminar, Data Science Disc...","[[Laboratory-Discussion, Lecture, Online], [La...","[SE, STAT, ENGL, ANSC, ACCY, PHIL, IS, ECE, TE...","[Biochemistry, Gender and Women's Studies, Neu...","[European Student Association, Social Sciences...","[Manager, Postsecondary Teacher, Social Worker...","[Psychology, Biology, Interdisciplinary Studie..."
3,Riyan,Gallea,European American or white,Female,Domestic,Middle income,[Visual],3.70,12,[General Social Sciences],"[Principles of Research, Oboe, Introduction to...","[[Laboratory, Lecture], [Laboratory, Lecture-D...","[MATH, ABE, ECON, ACCY, ECE, CW, ESL, VCM, DAN...","[Education, Sociology, Latina/Latino Studies, ...","[Social Work Club, Education Society, Latino S...","[Residential Advisor, Secondary School Teacher...","[Family Studies, Sociology, Social Sciences, E..."
4,Ulrich,Borths,European American or white,Nonbinary,Domestic,Lower-middle income,[Auditory],3.93,7,[Actuarial Science],"[General Chemistry Lab II, Modern Europe and t...","[[Laboratory-Discussion], [Lecture, Laboratory...","[STAT, CMN, BADM, ACCY, PHYS, BIOE, THEA, AE, ...","[Statistics, Theatre]","[Business Club, Mathematics Club, Agricultural...","[Mathematical Science Occupation, Management A...","[Statistics, Management, Calculus, Biostatisti..."


Notice that we have columns that are lists and columns that are strings. Machine learning models need the input data to be numerical. Thus some data preprocessing is required.

We import the Preprocessing class to do the preprocessing work.

In [9]:
from data_preprocessing.preprocessing import PreProcessing

Inside the Preprocessing class there are two functions that do the main preprocessing work:
- 'stringlist_to_binarylist': converts lists of strings into a binary list
- 'string_list_to_numberedlist': converts lits of strings into a numbered list.

Imagine the full options available are ['alice', 'bob', 'charlie']
Thus for the entry ['alice', 'charlie'] we get:
[1,0,1] for 'stringlist_to_binarylist'
[0,2] for 'string_list_to_numberedlist'

When we instantiate the class and call the 'preprocess_dataset' function both of the above functions will be called on certain columns. 'stringlist_to_binarylist' is called on 'learning styles' and 'string_list_to_numberedlist' is called on all the other lists.

In [10]:
preprocessor = PreProcessing(data)
start_time = time.time()
preprocessed_data = preprocessor.preprocess_dataset(synthetic_data)
end_time = time.time()
runtime = end_time - start_time
print(runtime)

0.14040303230285645


'preprocess_dataset' outputs a pandas dataframe. Lets look at the top 5 elements of the dataframe.

In [11]:
preprocessed_data.head(n=5) # Change n to larger numbers to see more rows of the dataframe

,learning style,gpa,student semester,major,previous courses,course types,course subjects,subjects of interest,extracurricular activities,career aspirations,future topics
0,"[0, 0, 0, 1]",3.90,12,[19],"[2041, 572, 1829, 1594, 3461, 2280, 3109, 3338...","[0, 4, 13, 5, 10, 1, 2, 3, 20, 19, 14, 17, 12]","[54, 63, 30, 127, 2, 125, 147, 207, 159, 15, 4...","[15, 10, 53, 18, 6]","[9, 70, 62, 102, 63, 116, 48, 185, 80]","[137, 70, 32, 27, 33, 85, 146, 51, 3, 82, 87, ...","[25, 24, 52, 122, 22]"
1,"[0, 0, 1, 0]",2.20,12,[41],"[2551, 3406, 2917, 2615, 1281, 1313, 3338, 346...","[9, 0, 4, 5, 10, 1, 2, 3, 7, 6, 20, 16, 17]","[54, 29, 163, 147, 106, 76, 90, 105, 48, 116, ...","[13, 27, 3]","[79, 75, 81, 178, 60, 112, 117]","[54, 4, 21, 60, 20, 14, 64, 82, 5, 53, 145, 40...","[68, 107, 170, 81, 26]"
2,"[0, 0, 0, 1]",2.27,5,"[92, 132]","[3338, 816, 1767, 1871, 1831, 1312, 253, 3205,...","[0, 5, 10, 1, 2, 3, 6]","[232, 160, 71, 15, 2, 146, 109, 64, 208, 56, 6...","[50, 72, 95, 6, 65, 4, 11, 18]","[22, 107, 99, 84, 71, 60, 86, 82]","[82, 112, 136, 16, 77, 130]","[169, 25, 213, 33, 114]"
3,"[1, 0, 0, 0]",3.70,12,[74],"[2527, 2340, 1832, 1067, 3453, 906, 983, 2494,...","[9, 0, 4, 5, 10, 1, 2, 3, 7, 11, 6, 18, 20, 16]","[127, 1, 65, 2, 64, 58, 79, 163, 61, 147, 106,...","[22, 11, 87, 37, 72, 0]","[120, 71, 44, 37, 107, 105, 121, 239, 80, 55]","[123, 130, 136, 114, 138, 26, 108, 50, 112, 14...","[65, 148, 191, 49, 147]"
4,"[0, 1, 0, 0]",3.93,7,[1],"[1313, 2236, 1916, 1690, 3461, 607, 816, 906, ...","[9, 0, 4, 13, 5, 10, 1, 2, 3, 7, 6, 8]","[160, 54, 30, 2, 147, 36, 162, 6, 48, 220, 56,...","[108, 124]","[60, 85, 81]","[86, 81, 2, 28, 37, 98, 0, 61, 25, 15, 140, 73...","[155, 106, 28, 23, 109]"


Now that the data has been preprocessed we must privatize the data to keep it safe.

We import the Privatizer class to do this.

In [12]:
from data_privatization.privatization import Privatizer

There are a variety of privatization methods you can try:
- Basic Differential Privacy (laplace noise addition)
- Uniform Noise Differential Privacy (uniform noise addition)
- Shuffling
Both Differential Privacy types can be done with or without list lengthening. This means the list columns like 'previous courses' could be lengthened according to the noise addition function. More details can be found in the README file. Let's try basic differential privacy with list lengthening.

In [13]:
privatization_type = 'basic differential privacy'
# Other 'privatization_type' options: 'uniform', 'shuffle', 'full shuffle' (full shuffle shuffles all of the rows)
privatizer = Privatizer(data, style=privatization_type, list_length=True)
# Can set 'list_length' to false if you don't want to allow the list sizes to change

Now we call 'privatize_dataset'. Use the time library to see how long the privatizer takes.

In [14]:
start_time = time.time()
privatized_data = privatizer.privatize_dataset(preprocessed_data)
end_time = time.time()
runtime = end_time - start_time
print(runtime)

0.026831865310668945


'preprocess_dataset' outputs a pandas dataframe. Lets look at the top 5 elements of the dataframe.

In [15]:
privatized_data.head(n=5) # Change n to larger numbers to see more rows of the dataframe

,learning style,gpa,student semester,major,previous courses,course types,course subjects,subjects of interest,extracurricular activities
0,"[0, 0, 0, 1]",3.13,6,[17],[3474],"[4, 19, 1]","[108, 96, 177, 219, 5, 193, 169, 147, 165, 189]","[3, 89, 48, 80, 0, 2, 52, 135, 137, 101, 73, 1...","[92, 4, 91, 9, 75, 114, 128, 281, 55, 136, 229..."
1,"[1, 0, 1, 0]",3.76,3,[],"[726, 2880, 3409, 1834, 1648, 367, 773, 551, 9...","[2, 21, 11, 6, 14, 6, 8, 0]","[201, 72, 90, 117, 53, 122, 19, 8, 227, 188, 9...","[100, 60, 126, 61, 10, 137, 50, 56, 13, 49, 13...","[67, 111]"
2,"[0, 0, 1, 0]",2.59,9,[],"[2440, 1987, 233, 1044, 1184, 695, 1936, 1502,...","[3, 8, 4, 10, 20, 19, 1, 17, 20, 5, 8, 19, 16]","[58, 216, 208, 148, 148, 179, 75]","[18, 112, 25, 106, 94, 6, 109, 9, 104, 1, 133,...","[230, 128, 11, 160, 169, 268, 203, 205, 52, 71..."
3,"[1, 0, 1, 0]",2.96,11,"[35, 125]","[2320, 2742, 731, 2154, 2317, 2897, 3057, 759,...","[17, 16, 20, 20, 2, 19, 0, 4]","[76, 81, 194, 191, 171, 41, 164, 181, 174, 77,...","[35, 66, 102, 89, 124, 139, 116, 28, 104, 82, ...",[]
4,"[1, 1, 0, 0]",2.92,4,"[24, 111]","[321, 2647, 1424, 2062, 993, 967, 293, 2080, 8...","[14, 6, 2, 4, 7, 5, 20, 9, 12, 20]","[142, 226, 217, 172, 209, 133, 88, 156, 169, 1...","[71, 30, 134, 114, 95, 85, 41, 127, 62, 54, 96...","[165, 100]"


We still have the problem of long lists. The 'previous courses list' can be over 30 elements long! Thus we call a new function from the Preprocessor class, 'create_RNN_models'. Three different recurrent neural network models are used to reduce the dimension of each list to 1 element. The three networks are: Simple, GRU (Gated Recurrent Units), and LSTM (Long Term Short Memory).

Since 'create_RNN_models' takes in a dataframe, there is no need to create a new instance of the Preprocessor class. Thus we should call:
- 'privatized_data': reduce dimensionality
- 'preprocessed_data': give a null for comparison at the end
- 'preprocessed_data' with 'utility=True': reduce dimensionality of the utility columns

Let's also calculate and compare the runtimes.

Let us just use the simple RNN dimensionality reduction. Though this can be switch by changing 'simple' to False and a different method to True.

In [19]:
# Only let one of simple, LSTM, and GRU be equal to true.

start_time = time.time()
privatized_data_reduced = preprocessor.create_RNN_models(privatized_data, simple=True, LSTM=False, GRU=False)
end_time = time.time()
runtime_pd = end_time - start_time

start_time = time.time()
nonprivatized_data_reduced = preprocessor.create_RNN_models(preprocessed_data, simple=True, LSTM=False, GRU=False)
end_time = time.time()
runtime_npd = end_time - start_time

start_time = time.time()
utility_cols_reduced = preprocessor.create_RNN_models(preprocessed_data, utility=True, simple=True, LSTM=False, GRU=False)
end_time = time.time()
runtime_uc = end_time - start_time

print(f'Privatized data runtime: {runtime_pd}')
print(f'Nonprivatized data runtime: {runtime_npd}')
print(f'Utility columns runtime: {runtime_uc}')

TypeError: PreProcessing.create_RNN_models() got an unexpected keyword argument 'simple'

'create_RNN_models' outputs a pandas dataframe. Lets look at the top 5 elements for each of the dataframes

In [17]:
print(privatized_data_reduced.head(n=5))
print(nonprivatized_data_reduced.head(n=5))
print(utility_cols_reduced.head(n=5))
# Change n to larger numbers to see more rows of the dataframe

AttributeError: 'list' object has no attribute 'head'

Now we have to preprocess the private columns so that machine learning models can train on them. To do this we import the PrivateColumns from the 'preprocessing_private_columns' file.

In [ ]:
from data_preprocessing.preprocessing_private_columns import PrivateColumns

Now we can instantiate the class.

In [ ]:
private_columns_data = PrivateColumns(synthetic_data)

Get the privacy columns from

In [ ]:
privacy_columns = private_cols.get_private_cols(df)

The reason for balancing the data privatization is to maximize the utility of the dataset while minimizing the privacy loss of the dataset. Perfectly private data would have no utility and vice versa.

Since our private columns are distinct classes, the privacy loss will be measured with accuracy where we want a low accuracy to keep the data safe. Meanwhile, after the RNNs our utility columns are essentially continuous. Thus utility gain will be measured with error where we want a low error to keep the data useful.

Our first test for this is the classifier decision tree. Since it is a classifier we will be using it to test privacy loss for the private columns. Import the DTClassifier class from 'decision_tree_classifier'.

In [12]:
from calculating_tradeoffs.decision_tree_classifier.decision_tree_classifier import DTClassifier

ModuleNotFoundError: No module named 'shap'

First we instantiate 2 classes. One for the privatized data and one for the non privatized data. We will only run them for the 'Simple1' RNN model to keep things simple. Ba Dum Tss! Similarly we will only look at the 'ethnoracial group' target.

In [ ]:
RNN_model = 'Simple1' # You can change it to 'GRU1' or 'LSTM1' if you like
target = 'ethnoracial group' # You can change it to 'gender', 'international status', or 'socioeconomic status' if you like

private_classifier = DTClassifier(privatization_type, RNN_model, target)
private_classifier.read_data(100)
private_classifier.split_data()

Now that we have instantiated the class we need to find the best model. We will use a metric called cost complexity pruning to decide how to prune our tree to prevent overfitting.

In [ ]:
best_model, ccp_alpha = private_classifier.get_best_model(make_graphs=False) # Leave this input to prevent the graphs from being produced
print(best_model)

What does this best model look like? Let's print out the decision tree to see for ourselves.

In [ ]:
private_classifier.plotter(model=best_model, show_fig=True)

Now that we have the best ccp alpha let us run that model and get the classification report.

In [ ]:
private_classifier.run_model(ccp_alpha=ccp_alpha, print_report=True, save_files=False, plot_files=False, get_shap=False) # Leave as is to prevent the graphs from being produced